# Mnist_nn
- 3개의 layer로 구현

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# Mnist dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers
linear1 = nn.Linear(784,256,bias=True)
linear2 = nn.Linear(256,256,bias=True)
linear3 = nn.Linear(256,10,bias=True)
relu = nn.ReLU()

In [7]:
# initialization
nn.init.normal_(linear1.weight)
nn.init.normal_(linear2.weight)
nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [8]:
# model
model = nn.Sequential(linear1,relu,
                      linear2,relu,
                      linear3).to(device)


In [9]:
# define loss/cost & optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [10]:
# total data 60,000 , batch_size = 100
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    for X,Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1,28*28).to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 129.308822632
Epoch: 0002 cost = 36.242008209
Epoch: 0003 cost = 23.013143539
Epoch: 0004 cost = 16.038034439
Epoch: 0005 cost = 11.614226341
Epoch: 0006 cost = 8.640940666
Epoch: 0007 cost = 6.390253067
Epoch: 0008 cost = 4.754397869
Epoch: 0009 cost = 3.557954073
Epoch: 0010 cost = 2.751285315
Epoch: 0011 cost = 2.162648678
Epoch: 0012 cost = 1.669728637
Epoch: 0013 cost = 1.252227783
Epoch: 0014 cost = 0.980631769
Epoch: 0015 cost = 0.799752474
Learning finished


94%의 정확도로 layer가 없는 모델보다 정확도가 더 증가하였다.

In [11]:
# test
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print(f"Accuarcy: {accuracy}")

    # Get one and predict
    r = random.randint(0,len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuarcy: 0.9453999996185303
Label:  9
Prediction:  9
